In [316]:
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from math import pi
# qiskit
from qiskit import (
    IBMQ, execute, transpile,
    QuantumRegister, ClassicalRegister, QuantumCircuit,
)
# qurry
sys.path.insert(0, './')
from backend import backendWrapper
from mori import TagMap
print("Modules import completed...")
print("-"*30+"\n ### Init IBMQ and Set up provider")
IBMQ.load_account()
IBMQ.providers()
provider = IBMQ.get_provider(
    hub='ibm-q-hub-ntu', group='ntu-internal', project='default')
backend = backendWrapper(provider)
print("IBMQ loading completed...")

Modules import completed...
------------------------------
 ### Init IBMQ and Set up provider


ibmqfactory.load_account:WARNING:2022-07-20 20:08:15,935: Credentials are already in use. The existing account in the session will be replaced.


IBMQ loading completed...


In [317]:
qaoaQurry = Qurry()
qaoaName = 'tsp'
problesSize = 2

nqubits = 3 ** 2
sampling = 10
shot = 2**(nqubits+6)
print(shot)

32768


In [318]:
from typing import Callable
from collections import defaultdict

def H_Ising(bitstring, Ising_coeff):
    spins = []
    for bit in bitstring:
        if bit == '0':
            spins.append(-1)
        elif bit == '1':
            spins.append(1)
    H = 0
    for i in range(0, nqubits):
        H += Ising_coeff[i][i]*spins[i]
        for j in range(i+1, nqubits):
            H += Ising_coeff[i][j]*spins[i]*spins[j]
    return H

def compute_expectation(counts: dict[str, int], Ising_coeff: np.ndarray):   
    """
    Computes expectation value based on measurement results
    """   
    sum = 0
    sum_count = 0
    for bitstring, count in counts.items():
        sum += count*H_Ising(bitstring, Ising_coeff)
        sum_count += count
    return sum/sum_count

from IPython.display import clear_output


# We will also bring the different circuit components that
# build the qaoa circuit under a single function
def create_qaoa_circ(
    Ising_coeff: np.ndarray, 
    theta: list[float],
) -> QuantumCircuit: 
    """
    Creates a parametrized qaoa circuit
    """   
    p = len(theta)//2  # number of alternating unitaries
    qc = QuantumCircuit(nqubits)
    
    beta = theta[:p]
    gamma = theta[p:]
    
    # initial_state
    for i in range(0, nqubits):
        qc.h(i)
    
    for irep in range(0, p):
        
        # problem unitary
        for i in range(nqubits):
            for j in range(nqubits):
                if Ising_coeff != 0 and i!=j:
                    qc.rzz(2 * Ising_coeff[i][j]*gamma[irep], i, j)
                    
        for i in range(nqubits):
            qc.rz(2 * Ising_coeff[i][i]*gamma[irep], i)
        
        # mixer unitary
        for i in range(nqubits):
            qc.rx(2 * beta[irep], i)
            
    qc.measure_all()
        
    return qc

def qaoa_circs(
    Ising_coeff: np.ndarray,
) -> Callable[[list[float]], QuantumCircuit]:
    
    def theta_depend(theta: list[float]) -> QuantumCircuit:
        return create_qaoa_circ(
            Ising_coeff=Ising_coeff, 
            theta=theta,
        )
    
    return theta_depend

def expect(
    Ising_coeff: np.ndarray,
) -> Callable[[dict[str, int]], float]:
    
    def counts_depend(counts: dict[str, int]) -> float:
        return compute_expectation(
            counts=counts, 
            Ising_coeff=Ising_coeff,
        )
    
    return counts_depend

def countCombine(countList: list[dict[str, int]]) -> dict[str, int]:
    summary = defaultdict(int)
    for c in countList:
        for k, v in c.items():
            summary[k] += v
    return summary

def expecting(Ising_matrix: np.ndarray) -> Callable[[list[float]], float]:
    
    qaoa_circs_giver = qaoa_circs(Ising_matrix)
    expectation_giver = expect(Ising_matrix)
    # Finally we write a function that executes the circuit on the chosen backend
    def get_expectation(theta: list[float]) -> float:
        """
        Runs parametrized circuit and get expectation value
    
        """
        # waveName = qaoaQurry.addWave(qaoa_circs_giver(theta))
        # singleIterResult = qaoaQurry.multiOutput(
        #     configList=[{ 
        #         'wave': waveName,
        #         'tags': waveName,
        #         'shots': 2**(nqubits+3),
        #         'sampling': sampling,
        #     }],
        #     backend=backend('aer_gpu'),
        #     saveLocation=Path('test.004')
        # )
        # countsList: list[dict[str, int]] = singleIterResult['tagMapCounts'][waveName]
        # counts = countCombine(countsList)
        
        qc = qaoa_circs_giver(theta)
        counts = backend('aer_gpu').run(qc, shots=2**16).result().get_counts()
        clear_output()
        print(counts)
        global ccc
        ccc = counts

        return expectation_giver(counts)
    
    return get_expectation

In [319]:
ccc = {}

In [320]:
aaaa = [[10616.601717798214,
  3535.533905932738,
  3535.533905932738,
  3535.533905932738,
  2.5,
  0.0,
  3535.533905932738,
  2.5,
  0.0],
 [3535.533905932738,
  10616.601717798214,
  3535.533905932738,
  0.0,
  3535.533905932738,
  2.5,
  0.0,
  3535.533905932738,
  2.5],
 [3535.533905932738,
  3535.533905932738,
  10616.601717798214,
  2.5,
  0.0,
  3535.533905932738,
  2.5,
  0.0,
  3535.533905932738],
 [3535.533905932738,
  2.5,
  0.0,
  10618.672785610077,
  3535.533905932738,
  3535.533905932738,
  3535.533905932738,
  3.5355339059327378,
  0.0],
 [0.0,
  3535.533905932738,
  2.5,
  3535.533905932738,
  10618.672785610077,
  3535.533905932738,
  0.0,
  3535.533905932738,
  3.5355339059327378],
 [2.5,
  0.0,
  3535.533905932738,
  3535.533905932738,
  3535.533905932738,
  10618.672785610077,
  3.5355339059327378,
  0.0,
  3535.533905932738],
 [3535.533905932738,
  2.5,
  0.0,
  3535.533905932738,
  3.5355339059327378,
  0.0,
  10618.672785610077,
  3535.533905932738,
  3535.533905932738],
 [0.0,
  3535.533905932738,
  2.5,
  0.0,
  3535.533905932738,
  3.5355339059327378,
  3535.533905932738,
  10618.672785610077,
  3535.533905932738],
 [2.5,
  0.0,
  3535.533905932738,
  3.5355339059327378,
  0.0,
  3535.533905932738,
  3535.533905932738,
  3535.533905932738,
  10618.672785610077]]

In [321]:
import torch

torch.manual_seed(0)
theta = torch.empty(int(nqubits)*2)
torch.nn.init.normal_(theta, mean=np.pi, std=0.01*np.pi)
theta = theta.tolist()
theta

[3.1062233448028564,
 3.1053903102874756,
 3.1593945026397705,
 3.1665215492248535,
 3.1604058742523193,
 3.092737913131714,
 3.130868673324585,
 3.1998066902160645,
 3.1562983989715576,
 3.1366381645202637,
 3.1361453533172607,
 3.1473569869995117,
 3.1852409839630127,
 3.1914288997650146,
 3.1713216304779053,
 3.1150877475738525,
 3.1708669662475586,
 3.181145668029785]

In [322]:
from scipy.optimize import minimize

expectation = expecting(Ising_matrix=aaaa)

res = minimize(expectation, 
                theta, options ={'iprint':1, 'disp':False, 'maxiter':10000,
                                 'catol':0.02, 'rhobeg':5.0},
            method='COBYLA')
res

{'111100100': 1, '110111110': 1, '111010010': 1, '111111011': 1, '111101011': 2, '111111100': 1, '011111101': 3, '011111110': 2, '011101110': 2, '111101110': 3, '001110111': 11, '111111000': 4, '001100111': 17, '011111100': 6, '111111010': 2, '001011011': 6, '111001010': 7, '100111010': 10, '101011010': 7, '001110101': 8, '111001111': 6, '110001111': 10, '111011001': 7, '100110101': 13, '101100111': 7, '110101101': 10, '101111001': 5, '101100101': 10, '111110101': 4, '011110111': 6, '011111011': 4, '101110110': 11, '111000111': 2, '110111101': 2, '110010101': 14, '000001111': 6, '101110101': 9, '011011011': 9, '101010101': 15, '101101101': 9, '111011110': 7, '110001110': 7, '001111011': 11, '011110110': 20, '110011101': 6, '101011110': 5, '011000111': 18, '001000111': 16, '010011101': 10, '011010111': 10, '110000111': 14, '101001110': 17, '000100111': 5, '011010011': 14, '111101100': 9, '111111110': 5, '010000111': 10, '110111001': 6, '110101011': 5, '100010111': 15, '011101010': 16, '

     fun: -33044.05561307062
   maxcv: 0.0
 message: 'Optimization terminated successfully.'
    nfev: 188
  status: 1
 success: True
       x: array([8.12517442, 3.08550383, 3.14024642, 3.15245134, 3.14174634,
       3.07551435, 3.08067349, 3.19738807, 3.13634318, 3.11177475,
       3.14161845, 3.14552228, 3.18612273, 3.17579729, 3.165465  ,
       3.11306482, 3.1745111 , 8.18569889])

In [ ]:
expectation

In [329]:
sum(ccc.values())

65536

In [324]:
def maxCounts(_counts: dict[str, int]) -> str:
    vs = list(_counts.values())
    vsMax = max(_counts.values())
    vsMaxIndex = list(_counts.keys())[vs.index(vsMax)]
    return vsMaxIndex

In [325]:
maxCounts(ccc)

'010100001'

In [326]:
ccc[maxCounts(ccc)]

1203

In [328]:
ccc

{'111100100': 1,
 '110111110': 1,
 '111010010': 1,
 '111111011': 1,
 '111101011': 2,
 '111111100': 1,
 '011111101': 3,
 '011111110': 2,
 '011101110': 2,
 '111101110': 3,
 '001110111': 11,
 '111111000': 4,
 '001100111': 17,
 '011111100': 6,
 '111111010': 2,
 '001011011': 6,
 '111001010': 7,
 '100111010': 10,
 '101011010': 7,
 '001110101': 8,
 '111001111': 6,
 '110001111': 10,
 '111011001': 7,
 '100110101': 13,
 '101100111': 7,
 '110101101': 10,
 '101111001': 5,
 '101100101': 10,
 '111110101': 4,
 '011110111': 6,
 '011111011': 4,
 '101110110': 11,
 '111000111': 2,
 '110111101': 2,
 '110010101': 14,
 '000001111': 6,
 '101110101': 9,
 '011011011': 9,
 '101010101': 15,
 '101101101': 9,
 '111011110': 7,
 '110001110': 7,
 '001111011': 11,
 '011110110': 20,
 '110011101': 6,
 '101011110': 5,
 '011000111': 18,
 '001000111': 16,
 '010011101': 10,
 '011010111': 10,
 '110000111': 14,
 '101001110': 17,
 '000100111': 5,
 '011010011': 14,
 '111101100': 9,
 '111111110': 5,
 '010000111': 10,
 '110111001

In [331]:
def filterCounts(counts: dict[str, int]) -> dict[str, int]:
    bitLen = len(list(counts)[0])
    qubitLen = int(np.sqrt(bitLen))
    if (qubitLen % 1 > 0):
        raise ValueError(f"bitLen: {bitLen}, qubitLen: {qubitLen}.")

    def filter(bitstr: str) -> bool:
        strList = [bitstr[qubitLen*(k):qubitLen*(k+1)] for k in range(qubitLen)]
        strList1Sum = [sum([int(b) for b in bs]) for bs in strList]
        strList1Pos = [[i for i, b in enumerate(bs) if b == '1'] for bs in strList]
        # print(strList1Pos)
        strList1PosOnly = all([len(lbs) == 1 for lbs in strList1Pos])
        # print(strList1Pos, strList1PosOnly)
        strList1PosImmutable = sum([sum(l) for l in strList1Pos]) == (qubitLen-1)*qubitLen/2
        # print(strList1PosImmutable)
            
        return all([ s==1 for s in strList1Sum]) and strList1PosImmutable
    
    filteredCounts = {
        k: v
    for k, v in counts.items() if filter(k) }
    
    return filteredCounts

sum(ccc.values()), sum(filterCounts(ccc).values()), sum(filterCounts(ccc).values())/sum(ccc.values())

(65536, 7107, 0.1084442138671875)